In [2]:
import os
import cv2
import numpy as np
import mediapipe as mp
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling1D
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Paths for train and test datasets
train_dir = r"S:\FYP\ASL Recognition\ASL_Dataset\Train"
test_dir = r"S:\FYP\ASL Recognition\ASL_Dataset\Test"

# Initialize Mediapipe hands model
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.9)

# Function to load and preprocess images from a directory
def load_data(data_dir):
    data = []
    labels = []

    for category in sorted(os.listdir(data_dir)):
        category_path = os.path.join(data_dir, category)
        if not os.path.isdir(category_path):
            continue

        for file_name in tqdm(os.listdir(category_path), desc=f"Processing {category}"):
            img_path = os.path.join(category_path, file_name)
            if not img_path.lower().endswith(('.png', '.jpg', '.jpeg')):
                continue

            img = cv2.imread(img_path)
            if img is None:
                continue

            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            results = hands.process(img_rgb)
            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    landmarks = []
                    for landmark in hand_landmarks.landmark:
                        landmarks.extend([landmark.x, landmark.y])
                    data.append(np.array(landmarks))
                    labels.append(category)
    return np.array(data), np.array(labels)


In [4]:

# Load train and test data
X_train, y_train = load_data(train_dir)
X_test, y_test = load_data(test_dir)

# Encode labels and convert to categorical format
le = LabelEncoder()
y_train_classes = le.fit_transform(y_train)  # Class indices for class weights
y_train = to_categorical(y_train_classes, num_classes=28)
y_test = to_categorical(le.transform(y_test), num_classes=28)  # Assumes the same classes are in test data

# Reshape input data for Conv1D
X_train = X_train.reshape(-1, 42, 1)
X_test = X_test.reshape(-1, 42, 1)


Processing A:   0%|          | 0/5863 [00:00<?, ?it/s]

c:\Users\zeafa\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
Processing Z: 100%|██████████| 4/4 [00:00<00:00, 17.51it/s]


In [5]:

# Calculate class weights to handle imbalance
class_weights = compute_class_weight('balanced', classes=np.unique(y_train_classes), y=y_train_classes)
class_weights = {i: class_weights[i] for i in range(len(class_weights))}

# Function for manual data augmentation
def augment_landmark_data(data, shift_range=0.02, noise_range=0.01):
    augmented_data = []
    for sample in data:
        # Apply random shift
        shift = np.random.uniform(-shift_range, shift_range, sample.shape)
        # Apply random noise
        noise = np.random.normal(0, noise_range, sample.shape)
        # Augment data
        augmented_sample = sample + shift + noise
        augmented_data.append(augmented_sample)
    return np.array(augmented_data)

# Perform manual data augmentation
X_train_augmented = augment_landmark_data(X_train, shift_range=0.02, noise_range=0.01)
X_train_combined = np.concatenate((X_train, X_train_augmented), axis=0)
y_train_combined = np.concatenate((y_train, y_train), axis=0)  # Duplicate labels




In [6]:
# Define optimized 1D CNN model
model = Sequential([
    Conv1D(64, kernel_size=3, activation='relu', input_shape=(42, 1)),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),

    Conv1D(128, kernel_size=5, activation='relu'),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),

    Conv1D(256, kernel_size=3, activation='relu'),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),

    GlobalAveragePooling1D(),  # Replaces Flatten for reduced parameters

    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(28, activation='softmax')  # 28 classes
])


c:\Users\zeafa\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),  # Adjustable learning rate
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Callbacks for training
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1)


In [8]:

history = model.fit(
    X_train_combined, y_train_combined,  # Use combined dataset
    validation_data=(X_test, y_test),
    epochs=5,
    class_weight=class_weights,  # Class weights for balance
    callbacks=[early_stopping, lr_scheduler],  # Callbacks for optimization
    verbose=1
)


Epoch 1/5
7770/7770 ━━━━━━━━━━━━━━━━━━━━ 111s 13ms/step - accuracy: 0.9170 - loss: 0.2851 - val_accuracy: 1.0000 - val_loss: 1.6559e-07 - learning_rate: 0.0010
Epoch 2/5
7770/7770 ━━━━━━━━━━━━━━━━━━━━ 120s 15ms/step - accuracy: 0.9957 - loss: 0.0176 - val_accuracy: 1.0000 - val_loss: 1.2269e-05 - learning_rate: 0.0010
Epoch 3/5
7770/7770 ━━━━━━━━━━━━━━━━━━━━ 138s 18ms/step - accuracy: 0.9970 - loss: 0.0151 - val_accuracy: 1.0000 - val_loss: 3.2143e-05 - learning_rate: 0.0010
Epoch 4/5
7770/7770 ━━━━━━━━━━━━━━━━━━━━ 142s 18ms/step - accuracy: 0.9974 - loss: 0.0144 - val_accuracy: 1.0000 - val_loss: 2.3190e-08 - learning_rate: 0.0010
Epoch 5/5
7770/7770 ━━━━━━━━━━━━━━━━━━━━ 143s 18ms/step - accuracy: 0.9979 - loss: 0.0094 - val_accuracy: 1.0000 - val_loss: 1.4892e-07 - learning_rate: 0.0010


In [12]:
accuracy = model.evaluate(X_train, y_train, verbose=1)
print(f"Train Accuracy: {accuracy}")


3885/3885 ━━━━━━━━━━━━━━━━━━━━ 21s 5ms/step - accuracy: 0.9994 - loss: 0.0019
Train Accuracy: [0.004854312166571617, 0.9985682368278503]


In [10]:

# Save the trained model for future use
model.save('optimized_asl_model.keras')

In [13]:
import cv2
# Real-time prediction with enhanced landmark drawing
cap = cv2.VideoCapture(0)
with mp_hands.Hands(min_detection_confidence=0.3, min_tracking_confidence=0.3) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.flip(frame, 1)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(frame_rgb)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                landmark_array = []
                for landmark in hand_landmarks.landmark:
                    landmark_array.extend([landmark.x, landmark.y])

                # Normalize and reshape for model input
                landmark_array = np.array(landmark_array).reshape(1, 42, 1)
                prediction = model.predict(landmark_array)
                predicted_class = le.inverse_transform([np.argmax(prediction)])[0]

                # Draw landmarks with adjusted size and style
                mp_drawing.draw_landmarks(
                    frame,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2, circle_radius=2),
                    mp_drawing.DrawingSpec(color=(0, 128, 255), thickness=2, circle_radius=1)
                )

                # Display prediction on frame
                cv2.putText(frame, f"Predicted: {predicted_class}", (20, 40),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

        cv2.imshow('Sign Language Recognition', frame)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


c:\Users\zeafa\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 258ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step


c:\Users\zeafa\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Users\zeafa\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step


c:\Users\zeafa\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Users\zeafa\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step


c:\Users\zeafa\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Users\zeafa\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step


c:\Users\zeafa\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Users\zeafa\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step


In [41]:
! pip install googletrans==4.0.0-rc1


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/55.1 kB ? eta -:--:--
   ---------------------------------------- 0.0/55.1 kB ? eta -:--:--
   ---------------------------------------- 0.0/55.1 kB ? eta -:--:--
   ---------------------------------------- 0.0/55.1 kB ? eta -:--:--
   ---------------------------------------- 0.0/55.1 kB ? eta -:--:--
   ---------------------------------------- 0.0/55.1 kB ? eta -:--:--
   ---------------------------------------- 0.0/55.1 kB ? eta -:--:--
   ---------------------------------------- 0.0/55.1 kB ? eta -:--:--
   ---------------------------------------- 0.0/55.1 kB ? eta -:--:--
   ---------------------------------------- 0.0/55.1 kB ? eta -:--:--
   ---------------------------------------- 0.0/55.1 kB ? eta -:--:--
   ---------------------------------------- 0.0/55.1 kB ? eta -:--:--
   ---------------------------------------- 0.0/55.1 kB ? 

In [ ]:
import os
import cv2
import numpy as np
import mediapipe as mp
from PIL import Image, ImageDraw, ImageFont

# Define the mapping from English letters to Urdu equivalents
english_to_urdu = {
    'A': 'الف', 'B': 'ب', 'C': 'پ', 'D': 'د', 'E': 'ے',
    'F': 'ف', 'G': 'گ', 'H': 'ہ', 'I': 'ی', 'J': 'ج',
    'K': 'ک', 'L': 'ل', 'M': 'م', 'N': 'ن', 'O': 'و',
    'P': 'پھ', 'Q': 'ق', 'R': 'ر', 'S': 'س', 'T': 'ت',
    'U': 'ؤ', 'V': 'و', 'W': 'و', 'X': 'خ', 'Y': 'ء', 'Z': 'ز'
}

# Load an Urdu font
font_path = r"S:\NotoNastaliqUrdu-Regular.ttf" # Update with the path to your Urdu font file
font = ImageFont.truetype(font_path, 32)

# Initialize Mediapipe hands model
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# Open video capture
cap = cv2.VideoCapture(0)
with mp_hands.Hands(min_detection_confidence=0.3, min_tracking_confidence=0.3) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.flip(frame, 1)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(frame_rgb)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                landmark_array = []
                for landmark in hand_landmarks.landmark:
                    landmark_array.extend([landmark.x, landmark.y])

                # Normalize and reshape for model input
                landmark_array = np.array(landmark_array).reshape(1, 42, 1)
                prediction = model.predict(landmark_array)
                predicted_class = le.inverse_transform([np.argmax(prediction)])[0]

                # Translate predicted English letter to Urdu
                urdu_letter = english_to_urdu.get(predicted_class, predicted_class)

                # Draw landmarks
                mp_drawing.draw_landmarks(
                    frame,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2, circle_radius=2),
                    mp_drawing.DrawingSpec(color=(0, 128, 255), thickness=2, circle_radius=1)
                )

                # Convert OpenCV frame to PIL Image for adding Urdu text
                pil_frame = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
                draw = ImageDraw.Draw(pil_frame)
                draw.text((20, 40), f"Urdu Letter: {urdu_letter}", font=font, fill=(255, 0, 0))

                # Convert back to OpenCV frame
                frame = cv2.cvtColor(np.array(pil_frame), cv2.COLOR_RGB2BGR)

        cv2.imshow('Sign Language Recognition with Urdu Translation', frame)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()
